<a href="https://colab.research.google.com/github/inspire007/Image-Recognition-using-CNN---Cats-and-dogs/blob/main/Image_recognition_using_CNN_Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as kImg
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
imgs = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
dataset = imgs.flow_from_directory('dataset/training_set', target_size=(64,64), batch_size=32, class_mode='binary')

test_imgs = ImageDataGenerator(rescale=1./255)
test_dataset = test_imgs.flow_from_directory('dataset/test_set', target_size=(64,64), batch_size=32, class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
cnn = Sequential([
    Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[64,64,3]),
    Dropout(0.5),
    MaxPool2D(pool_size=2, strides=2),
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

In [ ]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
cnn.fit(x = dataset, validation_data=test_dataset, epochs=25)

Epoch 1/25
250/250 [==============================] - 51s 201ms/step - loss: 0.6955 - accuracy: 0.5462 - val_loss: 0.6722 - val_accuracy: 0.5955
Epoch 2/25
250/250 [==============================] - 50s 201ms/step - loss: 0.6602 - accuracy: 0.6110 - val_loss: 0.6559 - val_accuracy: 0.6300
Epoch 3/25
250/250 [==============================] - 50s 198ms/step - loss: 0.6425 - accuracy: 0.6335 - val_loss: 0.6315 - val_accuracy: 0.6625
Epoch 4/25
250/250 [==============================] - 52s 206ms/step - loss: 0.6159 - accuracy: 0.6611 - val_loss: 0.6064 - val_accuracy: 0.6765
Epoch 5/25
250/250 [==============================] - 62s 249ms/step - loss: 0.5986 - accuracy: 0.6765 - val_loss: 0.5807 - val_accuracy: 0.7220
Epoch 6/25
250/250 [==============================] - 64s 257ms/step - loss: 0.5774 - accuracy: 0.6916 - val_loss: 0.5709 - val_accuracy: 0.7130
Epoch 7/25
250/250 [==============================] - 49s 193ms/step - loss: 0.5587 - accuracy: 0.7124 - val_loss: 0.5528 - val_ac

In [ ]:
cnn.save('cat_dog_model.h5')

In [ ]:
#cnn = tf.keras.models.load_model('cat_dog_model.h5')
#cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#single prediction
def pred_img(path):
    img = kImg.load_img(path, target_size=(64,64))
    img = kImg.img_to_array(img)
    #img = img/255
    img = np.expand_dims(img, axis=0)
    pred = cnn.predict(img)
    print(pred)
    if pred[0][0] > 0.5:
        return 'Dog'
    else:
        return 'Cat'

In [ ]:
for image in os.listdir('test'):
    path = os.path.join('test', image)
    pred = pred_img(path)
    print(f'File {path}: {pred}')

1/1 [==============================] - 0s 87ms/step
[[1.]]
File test\cat3.jpg: Dog
1/1 [==============================] - 0s 21ms/step
[[1.]]
File test\cat4.jpg: Dog
1/1 [==============================] - 0s 17ms/step
[[1.]]
File test\cat5.jpg: Dog
1/1 [==============================] - 0s 29ms/step
[[1.]]
File test\cat6.jpg: Dog
1/1 [==============================] - 0s 29ms/step
[[0.]]
File test\cat7.jpg: Cat
1/1 [==============================] - 0s 21ms/step
[[1.]]
File test\dog2.jpg: Dog
1/1 [==============================] - 0s 23ms/step
[[1.]]
File test\dog3.jpg: Dog
